In [ ]:
import pykoop
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from locomotion.envs.gym_envs import A1GymEnv
import numpy as np
from tqdm import tqdm

In [ ]:
def generate_data(num_episodes, len_episode):
    env = A1GymEnv(action_limit=(0.75, 0.75, 0.75), render=False, on_rack=False)

    X_train = []

    for episode in tqdm(range(num_episodes)):
        st = env.reset()
        for t in range(len_episode):
            at = env.action_space.sample()
            st_1, reward, done, info = env.step(at)

            X_train.append(np.concatenate(([episode], st, at), axis=0))

            st = st_1

    X_train = np.array(X_train)
    env.close()

    return X_train

In [ ]:
kp = pykoop.KoopmanPipeline(
    lifting_functions=[
        ('ma', pykoop.SkLearnLiftingFn(MaxAbsScaler())),
        ('pl', pykoop.PolynomialLiftingFn(order=3)),
        ('ss', pykoop.SkLearnLiftingFn(StandardScaler())),
    ],
    regressor=pykoop.Edmd(alpha=1),
)

for i in range(10):
    X_train = generate_data(1, 10)
    kp.fit(
        X_train,
        n_inputs=12,
        episode_feature=True,
    )
    X_val = generate_data(10, 100)
    # Equation Error
    X_pred = kp.predict(X_val)
    ee = np.sum((X_val[:, 1:20] - X_pred[:, 1:])**2)/(X_pred.shape[0])

    # Simulation Error
    X_pred = kp.predict_trajectory(X_val)
    se = np.sum((X_val[:, 1:20] - X_pred[:, 1:])**2)/(X_pred.shape[0])

    print(ee, se)